# Ausarbeitung von Funktionen

## Herzfrequenz aus EKG-Signal berechnen    

### Ausgangsdaten

- Zunächst müssen wir herausfinden, in welcher Struktur die Daten vorliegen und wie wir diese in Zukunft benutzen wollen
- Hierbei bietet es sich an die Daten immer als DataFrame zu behandeln, um die Vorteile von Pandas nutzen zu können

In [1]:
import pandas as pd

df = pd.read_csv(r'data/ekg_data/01_Ruhe.txt', sep='\t', header=None, names=['EKG in mV','Time in ms',])

df.head()

,EKG in mV,Time in ms
0,309,13666
1,307,13668
2,306,13670
3,304,13672
4,303,13674


### Visualisierung der Daten

- Plotly bietet sich an, um die Daten zu visualisieren

In [2]:
import plotly.express as px

fig = px.line(df, x='Time in ms', y='EKG in mV')
fig.show()

Es is sinnvoll sich zunächst nur ein Teil der Daten zu nutzen, um die Funktionen zu entwickeln und zu testen.

In [3]:
df_short = df.iloc[0:3000]

fig = px.line(df_short, x='Time in ms', y='EKG in mV')
fig.show()

### Eigenschaften der Find Peaks Funktion

- Wir wollen nur die R-Zacken finden (höchster Punkt im EKG-Signal)
- Hierfür könnten wir z.B. einen Threshold definieren, ab dem ein Peak als solcher erkannt wird
- Wir können nicht einfach Peaks über einem Threshold nutzen, sondern müssen uns auch das Maximum in einem bestimmten Zeitfenster anschauen
- Wir können die Rückgabe der Funktion auf verschiedene Arten gestalten, z.B. neuer Dataframe, der nur die Peaks enthält, als neue Spalte `is_peak` im alten Dataframe oder als Liste von Peaks, die z.B. die Indizes der Peaks enthält

#### Alle Peaks

- Ein Peak ist ein lokales Maximum, das größer ist als seine Nachbarn
- Um diese zu finden können wir über das Signal iterieren und schauen, ob der aktuelle Wert größer ist als seine Nachbarn
- Die Ergebnisse speichern wir in einer Liste

In [4]:
# Liste für Indexe der Peaks
peaks = []

# Variablen für die letzten drei Werte
last = 0
current = 0
next = 0

# Iteration über die Zeilen des DataFrames
for index, row in df_short.iterrows():
    last = current
    current = next
    next = row["EKG in mV"]

    # Wenn der aktuelle Wert größer als der vorherige und der nächste Wert ist, dann ist es ein Peak
    if last < current and current > next:
        # Wir passen den Index an, da wir schon beim nächsten Wert sind
        peaks.append(index-1)

peaks

[0,
 96,
 154,
 156,
 319,
 442,
 521,
 885,
 951,
 1188,
 1192,
 1323,
 1540,
 1656,
 1899,
 1921,
 2239,
 2241,
 2271,
 2341,
 2347,
 2603,
 2692,
 2720,
 2722,
 2754,
 2943]

##### Ergebnis

Offensichtlich ist die Funktion nicht in der Lage, die Peaks zu finden, da sich sich auch Plateaus in den Peaks befinden.

In [5]:
df_short.loc[:, "is_peak"] = False
df_short.loc[peaks, "is_peak"] = True

fig = px.scatter(df_short, x='Time in ms', y='EKG in mV', color='is_peak')
fig.show()

C:\Users\paul\AppData\Local\Temp\ipykernel_13884\3933025545.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



#### Anpassung durch Resampling

- Wir könnten die Funktion so anpassen, dass sie auch Plateaus in den Peaks erkennt
- Wenn wir das `>` Zeichen durch das `>=` Zeichen ersetzen, dann finden wir auch Plateaus, allerdings mehrere Punkte pro Plateau
- Alternativ können wir die Funktion auch resamplen, um die die Plateaus zu entfernen
- Mit etwas testen finden wir heraus, dass das Resampling mit einem Faktor von 5 gut funktioniert

In [7]:
df_short_resampled = df_short.iloc[::5]

fig = px.scatter(df_short_resampled, x='Time in ms', y='EKG in mV')
fig.show()

In [8]:
# Liste für Indexe der Peaks
peaks = []

# Variablen für die letzten drei Werte
last = 0
current = 0
next = 0

# Iteration über die Zeilen des DataFrames
for index, row in df_short_resampled.iterrows():
    last = current
    current = next
    next = row["EKG in mV"]
    
    # Wenn der aktuelle Wert größer als der vorherige und der nächste Wert ist, dann ist es ein Peak
    if last < current and current > next:
        # Wir passen den Index an, da wir schon beim nächsten Wert sind
        # VORSICHT hier müssen wir jeden 5. Wert nehmen, also müssen wir den Index anpassen
        peaks.append(index-5)

print(peaks)

df_short_resampled.loc[:, "is_peak"] = False
df_short_resampled.loc[peaks, "is_peak"] = True

fig = px.scatter(df_short_resampled, x='Time in ms', y='EKG in mV', color='is_peak')
fig.show()

[0, 320, 660, 785, 945, 1005, 1130, 1355, 1480, 1710, 2055, 2345, 2525, 2695, 2755]


#### Anpassung durch Threshold

- Immer noch werden zu viele Peaks gefunden, da auch kleine Peaks gefunden werden
- Wir können die Peaks auch filtern, indem wir nur Peaks finden, die größer als ein bestimmter Threshold sind
- Sinnvoll ist es, wenn wir den Threshold möglicht vor dem finden der Peaks definieren, um die Funktion schneller zu machen


In [9]:
# Folgenden Filter müssen wir nun in die Funktion einbauen
df_short_resampled = df_short_resampled[df_short_resampled["EKG in mV"]>340]
df_short_resampled

,EKG in mV,Time in ms,is_peak
315,367,14306,False
320,377,14316,True
325,369,14326,False
330,354,14336,False
655,355,14996,False
660,379,15006,True
665,376,15016,False
670,361,15027,False
675,344,15037,False
1000,358,15697,False


### Definition der Funktion

- Wir definieren die Funktion `find_peaks`, die ein EKG-Signal als Input nimmt und die Peaks zurückgibt
- Die Funktion hat die Parameter `signal` und `threshold`
- Die Funktion gibt die Peaks als Liste zurück
- Wir wollen die Funktion so schreiben, dass sie auch für andere Signale funktioniert, z.B. für ein PPG-Signal und der Dataframe auch andere Spalten enthält

In [72]:
def find_peaks(series, threshold, respacing_factor=5):
    """
    A function to find the peaks in a series
    Args:
        - series (pd.Series): The series to find the peaks in
        - threshold (float): The threshold for the peaks
        - respacing_factor (int): The factor to respace the series
    Returns:
        - peaks (list): A list of the indices of the peaks
    """
    # Respace the series
    series = series.iloc[::respacing_factor]
    
    # Filter the series
    series = series[series>threshold]


    peaks = []
    last = 0
    current = 0
    next = 0

    for index, row in series.items():
        last = current
        current = next
        next = row

        if last < current and current > next and current > threshold:
            peaks.append(index-respacing_factor)

    return peaks

In [73]:
df = pd.read_csv(r'data/ekg_data/01_Ruhe.txt', sep='\t', header=None, names=['EKG in mV','Time in ms',])

peaks = find_peaks(df["EKG in mV"].copy(), 340, 5)

peaks[0:5]

[320, 660, 1005, 1355, 1710]

In [74]:
df.loc[:, "is_peak"] = False
df.loc[peaks, "is_peak"] = True

In [75]:
fig = px.scatter(df.iloc[0:5000], x='Time in ms', y='EKG in mV', color='is_peak')
fig.show()

c:\code\programmieruebung2\.venv\Lib\site-packages\plotly\express\_core.py:2065: FutureWarning:

When grouping with a length-1 list-like, you will need to pass a length-1 tuple to get_group in a future version of pandas. Pass `(name,)` instead of `name` to silence this warning.



### Fazit

- Wir haben eine Funktion geschrieben, die Peaks in einem EKG-Signal findet
- Die Funktion ist nicht perfekt, aber sie ist ein guter Anfang
- Sie kann auch für andere Signale genutzt werden
- Wir können Die Funktion nun in ein Modul auslagern und in anderen Notebooks nutzen